## Creative Map using Geospatial Data
### Data Source: eGRID

In [25]:
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import LinearColormap

In [26]:
# Read eGRID data into a dataframe
egrid = pd.read_excel('../data/eGRID_ST21.xlsx', sheet_name='ST21')
egrid.head()

,State abbreviation,State annual CO2 equivalent emissions (tons),Latitude,Longitude
0,AK,3.050448e+06,61.2181,149.9003
1,AL,5.384399e+07,32.3792,86.3077
2,AR,3.339556e+07,34.7465,92.2896
3,AZ,3.947377e+07,33.4484,112.0740
4,CA,4.731524e+07,38.5816,121.4944


In [27]:
# Define a custom color scale for CO2 emissions using green to red colormap
co2_emissions_colormap = LinearColormap(
    ["#00ff00", "#ffff00", "#ff9900", "#ff0000"],
    vmin=egrid["State annual CO2 equivalent emissions (tons)"].min(),
    vmax=egrid["State annual CO2 equivalent emissions (tons)"].max(),
)

# Map emission value to a color
def get_marker_color(emissions):
    return co2_emissions_colormap(emissions)

# Map emission value to a size
def get_marker_size(emissions):
    min_size = 10
    max_size = 100
    min_emissions = egrid["State annual CO2 equivalent emissions (tons)"].min()
    max_emissions = egrid["State annual CO2 equivalent emissions (tons)"].max()
    normalized_emissions = (emissions - min_emissions) / (max_emissions - min_emissions)
    return min_size + normalized_emissions * (max_size - min_size)

In [ ]:
# Load the shapefile of US states boundaries
gdf = gpd.read_file("../data/tl_rd22_us_state/tl_rd22_us_state.shp")

# Create an interactive folium map
m = folium.Map(location=[37, -95], zoom_start=4)

# Add a choropleth layer to the map
folium.Choropleth(
    geo_data=gdf,
    data=egrid,
    columns=["State abbreviation", "State annual CO2 equivalent emissions (tons)"],
    fill_color=None,
    fill_opacity=0.1,
    line_opacity=0.1,
    legend_name="State annual CO2 equivalent emissions (tons)",
    highlight=True,
    name="CO2 emissions",
).add_to(m)

# Loop through the 'egrid' data to add circle markers to the map
for index, row in egrid.iterrows():
    state = row["State abbreviation"]
    co2_emissions = row["State annual CO2 equivalent emissions (tons)"]
    formatted_co2_emissions = "{:,.0f}".format(co2_emissions)

    # Get the coordinates for the state's centroid from the GeoDataFrame
    state_centroid = gdf.loc[gdf["STUSPS"] == state, "geometry"].centroid.iloc[0]

    # Calculate the marker size based on the CO2 emissions value
    marker_size = get_marker_size(co2_emissions)

    # Create a circle marker for each state
    marker = folium.CircleMarker(
        location=[state_centroid.y, state_centroid.x],
        radius=marker_size / 2,
        tooltip=f"{state}: {formatted_co2_emissions} tons",
        color=get_marker_color(co2_emissions),
        fill=True,
        fill_color=get_marker_color(co2_emissions),
        fill_opacity=0.6,
    ).add_to(m)

    # Add a color bar
    co2_emissions_colormap.add_to(m)

# Display the map
m

In [29]:
# Save the map as an HTML file
m.save("CreativeMap_eGRID_CO2.html")